In [9]:
import os
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# Activation functions
def relu(x):
    """
    ReLU activation: max(0, x)
    """
    assert isinstance(x, np.ndarray), "Input to ReLU must be a numpy array"
    result = np.maximum(0, x)
    assert np.all(result >= 0), "ReLU output must be non-negative"
    return result

def relu_derivative(x):
    """
    Derivative of ReLU: 1 if x > 0, else 0
    """
    assert isinstance(x, np.ndarray), "Input to ReLU derivative must be a numpy array"
    result = np.where(x > 0, 1, 0)
    assert np.all((result == 0) | (result == 1)), "ReLU derivative must be 0 or 1"
    return result

def softmax(x):
    """
    Softmax activation: exp(x) / sum(exp(x))
    """
    assert isinstance(x, np.ndarray), "Input to softmax must be a numpy array"
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))  # Numerical stability
    result = exp_x / np.sum(exp_x, axis=1, keepdims=True)
    assert np.all((result >= 0) & (result <= 1)), "Softmax output must be in [0, 1]"
    assert np.allclose(np.sum(result, axis=1), 1), "Softmax output must sum to 1 per sample"
    return result

# Neural Network class
class MultiClassNeuralNetwork:
    def __init__(self, layer_sizes, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        """
        Initialize the neural network with given layer sizes and learning rate.
        layer_sizes: List of integers [input_size, hidden1_size, ..., output_size]
        Uses Adam optimizer parameters: beta1, beta2, epsilon
        """
        assert isinstance(layer_sizes, list) and len(layer_sizes) >= 2, "layer_sizes must be a list with at least 2 elements"
        assert all(isinstance(size, int) and size > 0 for size in layer_sizes), "All layer sizes must be positive integers"
        assert isinstance(learning_rate, (int, float)) and learning_rate > 0, "Learning rate must be a positive number"
        
        self.layer_sizes = layer_sizes
        self.learning_rate = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.weights = []
        self.biases = []
        
        # Initialize weights and biases
        np.random.seed(42)
        for i in range(len(layer_sizes) - 1):
            w = np.random.randn(layer_sizes[i], layer_sizes[i+1]) * 0.01
            b = np.zeros((1, layer_sizes[i+1]))
            assert w.shape == (layer_sizes[i], layer_sizes[i+1]), f"Weight matrix {i+1} has incorrect shape"
            assert b.shape == (1, layer_sizes[i+1]), f"Bias vector {i+1} has incorrect shape"
            self.weights.append(w)
            self.biases.append(b)
        
        # Initialize Adam moment estimates
        self.m_weights = [np.zeros_like(w) for w in self.weights]
        self.v_weights = [np.zeros_like(w) for w in self.weights]
        self.m_biases = [np.zeros_like(b) for b in self.biases]
        self.v_biases = [np.zeros_like(b) for b in self.biases]
        self.t = 0  # Time step for Adam

    def forward(self, X):
        """
        Forward propagation: Z^{[l]} = A^{[l-1]} W^{[l]} + b^{[l]}, A^{[l]} = g(Z^{[l]})
        """
        assert isinstance(X, np.ndarray), "Input X must be a numpy array"
        assert X.shape[1] == self.layer_sizes[0], f"Input dimension ({X.shape[1]}) must match input layer size ({self.layer_sizes[0]})"
        
        self.activations = [X]
        self.z_values = []
        
        for i in range(len(self.weights) - 1):
            z = self.activations[-1] @ self.weights[i] + self.biases[i]
            assert z.shape == (X.shape[0], self.layer_sizes[i+1]), f"Z^{[i+1]} has incorrect shape"
            self.z_values.append(z)
            self.activations.append(relu(z))
        
        z = self.activations[-1] @ self.weights[-1] + self.biases[-1]
        assert z.shape == (X.shape[0], self.layer_sizes[-1]), "Output Z has incorrect shape"
        self.z_values.append(z)
        output = softmax(z)
        assert output.shape == (X.shape[0], self.layer_sizes[-1]), "Output A has incorrect shape"
        self.activations.append(output)
        
        return self.activations[-1]

    def compute_loss(self, y_true, y_pred):
        """
        Categorical Cross-Entropy: J = -1/m * sum(y_true * log(y_pred))
        """
        assert isinstance(y_true, np.ndarray) and isinstance(y_pred, np.ndarray), "Inputs to loss must be numpy arrays"
        assert y_true.shape == y_pred.shape, "y_true and y_pred must have the same shape"
        
        y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
        loss = -np.mean(np.sum(y_true * np.log(y_pred), axis=1))
        assert not np.isnan(loss), "Loss computation resulted in NaN"
        return loss

    def compute_accuracy(self, y_true, y_pred):
        """
        Compute accuracy: proportion of correct predictions
        """
        assert isinstance(y_true, np.ndarray) and isinstance(y_pred, np.ndarray), "Inputs to accuracy must be numpy arrays"
        assert y_true.shape == y_pred.shape, "y_true and y_pred must have the same shape"
        
        predictions = np.argmax(y_pred, axis=1)
        true_labels = np.argmax(y_true, axis=1)
        accuracy = np.mean(predictions == true_labels)
        assert 0 <= accuracy <= 1, "Accuracy must be between 0 and 1"
        return accuracy

    def backward(self, X, y, outputs):
        """
        Backpropagation: compute dW^{[l]}, db^{[l]} for each layer with Adam optimizer
        """
        assert isinstance(X, np.ndarray) and isinstance(y, np.ndarray) and isinstance(outputs, np.ndarray), "Inputs to backward must be numpy arrays"
        assert X.shape[1] == self.layer_sizes[0], f"Input dimension ({X.shape[1]}) must match input layer size ({self.layer_sizes[0]})"
        assert y.shape == outputs.shape, "y and outputs must have the same shape"
        
        m = X.shape[0]
        self.d_weights = [np.zeros_like(w) for w in self.weights]
        self.d_biases = [np.zeros_like(b) for b in self.biases]
        
        dZ = outputs - y  # Gradient for softmax + cross-entropy
        assert dZ.shape == outputs.shape, "dZ for output layer has incorrect shape"
        self.d_weights[-1] = (self.activations[-2].T @ dZ) / m
        self.d_biases[-1] = np.sum(dZ, axis=0, keepdims=True) / m
        
        for i in range(len(self.weights) - 2, -1, -1):
            dZ = (dZ @ self.weights[i+1].T) * relu_derivative(self.z_values[i])
            assert dZ.shape == (X.shape[0], self.layer_sizes[i+1]), f"dZ^{[i+1]} has incorrect shape"
            self.d_weights[i] = (self.activations[i].T @ dZ) / m
            self.d_biases[i] = np.sum(dZ, axis=0, keepdims=True) / m
        
        # L2 regularization
        l2_lambda = 0.01
        for i in range(len(self.weights)):
            self.d_weights[i] += l2_lambda * self.weights[i] / m
        
        # Adam optimizer updates
        self.t += 1
        for i in range(len(self.weights)):
            # Update biased first moment estimate
            self.m_weights[i] = self.beta1 * self.m_weights[i] + (1 - self.beta1) * self.d_weights[i]
            self.m_biases[i] = self.beta1 * self.m_biases[i] + (1 - self.beta1) * self.d_biases[i]
            # Update biased second moment estimate
            self.v_weights[i] = self.beta2 * self.v_weights[i] + (1 - self.beta2) * (self.d_weights[i] ** 2)
            self.v_biases[i] = self.beta2 * self.v_biases[i] + (1 - self.beta2) * (self.d_biases[i] ** 2)
            # Bias-corrected moments
            m_hat_w = self.m_weights[i] / (1 - self.beta1 ** self.t)
            m_hat_b = self.m_biases[i] / (1 - self.beta1 ** self.t)
            v_hat_w = self.v_weights[i] / (1 - self.beta2 ** self.t)
            v_hat_b = self.v_biases[i] / (1 - self.beta2 ** self.t)
            # Update weights and biases
            self.weights[i] -= self.learning_rate * m_hat_w / (np.sqrt(v_hat_w) + self.epsilon)
            self.biases[i] -= self.learning_rate * m_hat_b / (np.sqrt(v_hat_b) + self.epsilon)

    def train(self, X, y, X_val, y_val, epochs, batch_size, patience=10):
        """
        Train the neural network using mini-batch SGD with validation and early stopping
        """
        assert isinstance(X, np.ndarray) and isinstance(y, np.ndarray), "X and y must be numpy arrays"
        assert isinstance(X_val, np.ndarray) and isinstance(y_val, np.ndarray), "X_val and y_val must be numpy arrays"
        assert X.shape[1] == self.layer_sizes[0], f"Input dimension ({X.shape[1]}) must match input layer size ({self.layer_sizes[0]})"
        assert y.shape[1] == self.layer_sizes[-1], f"Output dimension ({y.shape[1]}) must match output layer size ({self.layer_sizes[-1]})"
        assert X_val.shape[1] == self.layer_sizes[0], f"Validation input dimension ({X_val.shape[1]}) must match input layer size ({self.layer_sizes[0]})"
        assert y_val.shape[1] == self.layer_sizes[-1], f"Validation output dimension ({y_val.shape[1]}) must match output layer size ({self.layer_sizes[-1]})"
        assert isinstance(epochs, int) and epochs > 0, "Epochs must be a positive integer"
        assert isinstance(batch_size, int) and batch_size > 0, "Batch size must be a positive integer"
        
        train_losses = []
        val_losses = []
        train_accuracies = []
        val_accuracies = []
        best_val_loss = float('inf')
        patience_counter = 0
        
        for epoch in range(epochs):
            indices = np.random.permutation(X.shape[0])
            X_shuffled = X[indices]
            y_shuffled = y[indices]
            epoch_loss = 0
            for i in range(0, X.shape[0], batch_size):
                X_batch = X_shuffled[i:i+batch_size]
                y_batch = y_shuffled[i:i+batch_size]
                outputs = self.forward(X_batch)
                epoch_loss += self.compute_loss(y_batch, outputs)
                self.backward(X_batch, y_batch, outputs)
            
            train_loss = epoch_loss / (X.shape[0] // batch_size)
            train_pred = self.forward(X)
            train_accuracy = self.compute_accuracy(y, train_pred)
            val_pred = self.forward(X_val)
            val_loss = self.compute_loss(y_val, val_pred)
            val_accuracy = self.compute_accuracy(y_val, val_pred)
            
            train_losses.append(train_loss)
            val_losses.append(val_loss)
            train_accuracies.append(train_accuracy)
            val_accuracies.append(val_accuracy)
            
            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, "
                      f"Train Acc: {train_accuracy:.4f}, Val Acc: {val_accuracy:.4f}")
            
            # Early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
            else:
                patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch}")
                break
        
        return train_losses, val_losses, train_accuracies, val_accuracies

    def predict(self, X):
        """
        Predict class labels
        """
        assert isinstance(X, np.ndarray), "Input X must be a numpy array"
        assert X.shape[1] == self.layer_sizes[0], f"Input dimension ({X.shape[1]}) must match input layer size ({self.layer_sizes[0]})"
        
        outputs = self.forward(X)
        predictions = np.argmax(outputs, axis=1)
        assert predictions.shape == (X.shape[0],), "Predictions have incorrect shape"
        return predictions

# Cross-validation function
def cross_validate(X, y, layer_sizes, learning_rate, epochs, batch_size, k=5):
    """
    Perform k-fold cross-validation
    """
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    val_accuracies = []
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        y_train_one_hot = one_hot_encoder.fit_transform(y_train.reshape(-1, 1))
        y_val_one_hot = one_hot_encoder.transform(y_val.reshape(-1, 1))
        nn = MultiClassNeuralNetwork(layer_sizes, learning_rate)
        _, _, _, val_accs = nn.train(X_train, y_train_one_hot, X_val, y_val_one_hot, epochs, batch_size)
        val_accuracies.append(val_accs[-1])
    print(f"Cross-validation accuracies: {val_accuracies}")
    print(f"Mean CV accuracy: {np.mean(val_accuracies):.4f} ± {np.std(val_accuracies):.4f}")
    return val_accuracies

# Define paths to dataset
train_dir = '/Users/macbook/Desktop/M2/Prof M. Benaddy Deep Learning/Tp 4/Tifinagh/Dataset/train_data'
test_dir = '/Users/macbook/Desktop/M2/Prof M. Benaddy Deep Learning/Tp 4/Tifinagh/Dataset/test_data'
print(f"Train directory: {train_dir}")
print(f"Test directory: {test_dir}")
print(f"Current working directory: {os.getcwd()}")

# Verify directories exist
if not os.path.exists(train_dir):
    raise FileNotFoundError(f"Train directory not found: {train_dir}")
if not os.path.exists(test_dir):
    raise FileNotFoundError(f"Test directory not found: {test_dir}")

# Build DataFrame from train_data folder structure
train_image_paths = []
train_labels = []
for label_dir in os.listdir(train_dir):
    label_path = os.path.join(train_dir, label_dir)
    if os.path.isdir(label_path):
        for img_name in os.listdir(label_path):
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                train_image_paths.append(os.path.join(label_dir, img_name))  # Relative path
                train_labels.append(label_dir)
train_df = pd.DataFrame({'image_path': train_image_paths, 'label': train_labels})

# Build DataFrame from test_data folder structure
test_image_paths = []
test_labels = []
for label_dir in os.listdir(test_dir):
    label_path = os.path.join(test_dir, label_dir)
    if os.path.isdir(label_path):
        for img_name in os.listdir(label_path):
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                test_image_paths.append(os.path.join(label_dir, img_name))  # Relative path
                test_labels.append(label_dir)
test_df = pd.DataFrame({'image_path': test_image_paths, 'label': test_labels})

# Verify DataFrames
assert not train_df.empty, "No training data loaded. Check train_data directory."
assert not test_df.empty, "No test data loaded. Check test_data directory."
print(f"Loaded {len(train_df)} training samples with {train_df['label'].nunique()} unique classes.")
print(f"Loaded {len(test_df)} test samples with {test_df['label'].nunique()} unique classes.")

# Encode labels
label_encoder = LabelEncoder()
train_df['label_encoded'] = label_encoder.fit_transform(train_df['label'])
test_df['label_encoded'] = label_encoder.transform(test_df['label'])
num_classes = len(label_encoder.classes_)
assert num_classes == 33, f"Expected 33 classes, got {num_classes}"

# Function to load and preprocess images
def load_and_preprocess_image(image_path, base_dir, target_size=(32, 32)):
    """
    Load and preprocess an image: convert to grayscale, resize, normalize
    """
    full_path = os.path.join(base_dir, image_path)
    assert os.path.exists(full_path), f"Image not found: {full_path}"
    img = cv2.imread(full_path, cv2.IMREAD_GRAYSCALE)
    assert img is not None, f"Failed to load image: {full_path}"
    img = cv2.resize(img, target_size)
    img = img.astype(np.float32) / 255.0  # Normalize
    return img.flatten()  # Flatten for MLP

# Load training and test images
X_train = np.array([load_and_preprocess_image(path, train_dir) for path in train_df['image_path']])
y_train = train_df['label_encoded'].values
X_test = np.array([load_and_preprocess_image(path, test_dir) for path in test_df['image_path']])
y_test = test_df['label_encoded'].values

# Verify dimensions
assert X_train.shape[0] == y_train.shape[0], "Mismatch between number of training images and labels"
assert X_test.shape[0] == y_test.shape[0], "Mismatch between number of test images and labels"
assert X_train.shape[1] == 32 * 32, f"Expected flattened image size of {32*32}, got {X_train.shape[1]}"
assert X_test.shape[1] == 32 * 32, f"Expected flattened image size of {32*32}, got {X_test.shape[1]}"

# Split training data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, stratify=y_train, random_state=42)

# Convert to NumPy arrays
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

print(f"Train: {X_train.shape[0]} samples, Validation: {X_val.shape[0]} samples, Test: {X_test.shape[0]} samples")

# One-hot encode labels
one_hot_encoder = OneHotEncoder(sparse_output=False)
y_train_one_hot = np.array(one_hot_encoder.fit_transform(y_train.reshape(-1, 1)))
y_val_one_hot = np.array(one_hot_encoder.transform(y_val.reshape(-1, 1)))
y_test_one_hot = np.array(one_hot_encoder.transform(y_test.reshape(-1, 1)))

# Verify one-hot arrays
assert isinstance(y_train_one_hot, np.ndarray), "y_train_one_hot must be a numpy array"
assert isinstance(y_val_one_hot, np.ndarray), "y_val_one_hot must be a numpy array"
assert isinstance(y_test_one_hot, np.ndarray), "y_test_one_hot must be a numpy array"

# Create and train the model
layer_sizes = [X_train.shape[1], 64, 32, num_classes]  # [1024, 64, 32, 33]
nn = MultiClassNeuralNetwork(layer_sizes, learning_rate=0.001)
train_losses, val_losses, train_accuracies, val_accuracies = nn.train(
    X_train, y_train_one_hot, X_val, y_val_one_hot, epochs=100, batch_size=32
)

# Cross-validation
X_temp = np.concatenate((X_train, X_val), axis=0)
y_temp = np.concatenate((y_train, y_val), axis=0)
cv_accuracies = cross_validate(X_temp, y_temp, layer_sizes, learning_rate=0.001, epochs=100, batch_size=32, k=5)

# Predictions and evaluation on test set
y_pred = nn.predict(X_test)
print("\nClassification Report (Test set):")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix (Test set)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.savefig('confusion_matrix.png')
plt.close()

# Loss and accuracy curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Loss curve
ax1.plot(train_losses, label='Train Loss')
ax1.plot(val_losses, label='Validation Loss')
ax1.set_title('Loss Curve')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()

# Accuracy curve
ax2.plot(train_accuracies, label='Train Accuracy')
ax2.plot(val_accuracies, label='Validation Accuracy')
ax2.set_title('Accuracy Curve')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.legend()

plt.tight_layout()
plt.savefig('loss_accuracy_plot.png')
plt.close()

Train directory: /Users/macbook/Desktop/M2/Prof M. Benaddy Deep Learning/Tp 4/Tifinagh/Dataset/train_data
Test directory: /Users/macbook/Desktop/M2/Prof M. Benaddy Deep Learning/Tp 4/Tifinagh/Dataset/test_data
Current working directory: /Users/macbook/Desktop/M2/Prof M. Benaddy Deep Learning/Tp 4/Tifinagh
Loaded 28182 training samples with 33 unique classes.
Loaded 28182 test samples with 33 unique classes.
Train: 21136 samples, Validation: 7046 samples, Test: 28182 samples
Epoch 0, Train Loss: 2.5060, Val Loss: 1.9582, Train Acc: 0.3913, Val Acc: 0.3849
Epoch 10, Train Loss: 0.5326, Val Loss: 0.6068, Train Acc: 0.8409, Val Acc: 0.8046
Epoch 20, Train Loss: 0.2597, Val Loss: 0.3841, Train Acc: 0.9303, Val Acc: 0.8754
Epoch 30, Train Loss: 0.1571, Val Loss: 0.3098, Train Acc: 0.9676, Val Acc: 0.9015
Epoch 40, Train Loss: 0.1173, Val Loss: 0.3094, Train Acc: 0.9706, Val Acc: 0.9070
Epoch 50, Train Loss: 0.0978, Val Loss: 0.2942, Train Acc: 0.9773, Val Acc: 0.9032
Early stopping at epoch 